# Border Box Regression using VGG-16

## Border Box - Training

#### Imports

In [1]:
import sys

sys.path.insert(0, '/home/indrap24/Desktop/Projects/VORTEX/vortex/utils')

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [3]:
# import the necessary packages
from utils import config
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Flatten, Dense, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os

In [ ]:
import pandas as pd
df = pd.read_csv("../../data/processed/Obj_Det/labels.csv")
df.drop(["width", "height", "class"], axis=1, inplace=True)
# save the dataframe
df.to_csv("../../data/processed/Obj_Det/labels.csv", index=False)

In [4]:
# load the contents of the CSV annotations file
print("[INFO] loading dataset...")
rows = open(config.ANNOTS_PATH).read().strip().split("\n")
# initialize the list of data (images), our target output predictions
# (bounding box coordinates), along with the filenames of the
# individual images
data = []
targets = []
filenames = []

[INFO] loading dataset...


In [5]:
# loop over the rows
for row in rows:
	# break the row into the filename and bounding box coordinates
	row = row.split(",")
	(filename, startX, startY, endX, endY) = row
	print(filename)
	# derive the path to the input image, load the image (in OpenCV format), and grab its dimensions
	imagePath = os.path.sep.join([config.IMAGES_PATH, filename])
	print(imagePath)
	image = cv2.imread(imagePath)
	(h, w) = image.shape[:2]
	# scale the bounding box coordinates relative to the spatial
	# dimensions of the input image
	startX = float(startX) / w
	startY = float(startY) / h
	endX = float(endX) / w
	endY = float(endY) / h
	# load the image and preprocess it
	image = load_img(imagePath, target_size=(224, 224))
	image = img_to_array(image)
	# update our list of data, targets, and filenames
	data.append(image)
	targets.append((startX, startY, endX, endY))
	filenames.append(filename)

3DIMG_29MAY2017_0500_L1B_STD_IR2.jpg
../../data/processed/Obj_Det/images/3DIMG_29MAY2017_0500_L1B_STD_IR2.jpg
3DIMG_15MAY2021_1930_L1B_STD_IR2.jpg
../../data/processed/Obj_Det/images/3DIMG_15MAY2021_1930_L1B_STD_IR2.jpg
3DIMG_15NOV2018_1030_L1B_STD_IR2.jpg
../../data/processed/Obj_Det/images/3DIMG_15NOV2018_1030_L1B_STD_IR2.jpg
3DIMG_30APR2019_0430_L1B_STD_IR2.jpg
../../data/processed/Obj_Det/images/3DIMG_30APR2019_0430_L1B_STD_IR2.jpg
3DIMG_11OCT2014_0230_L1B_STD_IR2.jpg
../../data/processed/Obj_Det/images/3DIMG_11OCT2014_0230_L1B_STD_IR2.jpg
3DIMG_16APR2017_1000_L1B_STD_IR2.jpg
../../data/processed/Obj_Det/images/3DIMG_16APR2017_1000_L1B_STD_IR2.jpg
3DIMG_08NOV2019_0900_L1B_STD_IR2.jpg
../../data/processed/Obj_Det/images/3DIMG_08NOV2019_0900_L1B_STD_IR2.jpg
3DIMG_02JUN2020_0830_L1B_STD_IR2.jpg
../../data/processed/Obj_Det/images/3DIMG_02JUN2020_0830_L1B_STD_IR2.jpg
3DIMG_11DEC2016_0530_L1B_STD_IR2.jpg
../../data/processed/Obj_Det/images/3DIMG_11DEC2016_0530_L1B_STD_IR2.jpg
3DIMG_11NO

In [6]:
# convert the data and targets to NumPy arrays, scaling the input
# pixel intensities from the range [0, 255] to [0, 1]
data = np.array(data, dtype="float32") / 255.0
targets = np.array(targets, dtype="float32")
# partition the data into training and testing splits using 90% of
# the data for training and the remaining 10% for testing
split = train_test_split(data, targets, filenames, test_size=0.20, train_size=0.80, random_state=42)
# unpack the data split
(trainImages, testImages) = split[:2]
(trainTargets, testTargets) = split[2:4]
(trainFilenames, testFilenames) = split[4:]
# write the testing filenames to disk so that we can use then
# when evaluating/testing our bounding box regressor
print("[INFO] saving testing filenames...")
f = open(config.TEST_FILENAMES, "w")
f.write("\n".join(testFilenames))
f.close()

[INFO] saving testing filenames...


In [8]:
# load the VGG16 network, ensuring the head FC layers are left off
vgg = VGG16(weights="imagenet", include_top=False,
	input_tensor=Input(shape=(224, 224, 3)))
# freeze all VGG layers so they will *not* be updated during the
# training process
vgg.trainable = False
# flatten the max-pooling output of VGG
flatten = vgg.output
flatten = Flatten()(flatten)
# construct a fully-connected layer header to output the predicted
# bounding box coordinates
bboxHead = Dense(128, activation="relu")(flatten)
bboxHead = Dense(64, activation="relu")(bboxHead)
bboxHead = Dense(32, activation="relu")(bboxHead)
bboxHead = Dense(4, activation="sigmoid")(bboxHead)
# construct the model we will fine-tune for bounding box regression
model = Model(inputs=vgg.input, outputs=bboxHead)

In [9]:
# initialize the optimizer, compile the model, and show the model
# summary
opt = Adam(lr=config.INIT_LR)
model.compile(loss="mse", optimizer=opt)
print(model.summary())
# train the network for bounding box regression
print("[INFO] training bounding box regressor...")
H = model.fit(
	trainImages, trainTargets,
	validation_data=(testImages, testTargets),
	batch_size=config.BATCH_SIZE,
	epochs=config.NUM_EPOCHS,
	verbose=1)

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

/home/indrap24/anaconda3/envs/vortex3.9/lib/python3.9/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)
2022-03-28 22:46:14.235298: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 411041792 exceeds 10% of free system memory.
2022-03-28 22:46:14.484935: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 411041792 exceeds 10% of free system memory.


 1/11 [=>............................] - ETA: 2:46 - loss: 0.1146

2022-03-28 22:46:29.652909: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 411041792 exceeds 10% of free system memory.
2022-03-28 22:46:30.110372: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 411041792 exceeds 10% of free system memory.


 2/11 [====>.........................] - ETA: 2:10 - loss: 0.0869

2022-03-28 22:46:44.183592: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 411041792 exceeds 10% of free system memory.


11/11 [==============================] - 157s 14s/step - loss: 0.0462 - val_loss: 0.0256
Epoch 2/5
11/11 [==============================] - 130s 12s/step - loss: 0.0245 - val_loss: 0.0185
Epoch 3/5
11/11 [==============================] - 134s 12s/step - loss: 0.0163 - val_loss: 0.0156
Epoch 4/5
11/11 [==============================] - 135s 12s/step - loss: 0.0125 - val_loss: 0.0137
Epoch 5/5
11/11 [==============================] - 134s 12s/step - loss: 0.0105 - val_loss: 0.0124
